In [ ]:
import fire
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import os
from docx import Document
from tqdm.auto import tqdm
import pandas as pd
import ast
import sqlite3
import warnings
import time
import re


In [ ]:
# Подключение к базе данных SQLite
conn = sqlite3.connect('buhpulse_data.sqlite')

# Создание курсора
cursor = conn.cursor()

# Запрос списка таблиц
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Получение результатов
tables = cursor.fetchall()

# Закрытие соединения с базой данных
conn.close()


In [ ]:
# Подключение к базе данных SQLite
conn = sqlite3.connect('buhpulse_data.sqlite')
df = pd.DataFrame()
for i in tqdm(tables):
    # Запрос данных из таблицы
    query = f"SELECT * FROM {i[0]}"
    
    # Чтение данных в DataFrame
    df_forum = pd.read_sql_query(query, conn)
    df_forum['messages'] = i[0]
    df = pd.concat([df, df_forum])
    
conn.close()
df.shape    

# Иcпользуемый prompt

In [ ]:
SYSTEM_PROMPT = """
Ты — эксперт в области бухгалтерии. Твоя задача — классифицировать темы, связанные с бухгалтерским учетом, налогами, отчетностью и т. д.
Дан текст из чата, который может содержать информацию, релевантную бухгалтерам или работе предприятия. Ваша задача — структурировать информацию, извлекая ключевые темы и детали.
Каждую полученную тему нужно отнести к одной из следующих категорий:
1. Налоги и налогообложение
2. Бухгалтерский учет
3. Финансовая отчетность
4. Внутренний контроль и аудит
5. Финансовое планирование и прогнозирование
6. Учет заработной платы и кадровое делопроизводство
7. Программное обеспечение для бухгалтерии и автоматизация
8. Взаимодействие с государственными органами
9. Правовые аспекты бухгалтерии и налогообложения
10. Страхование и управление финансовыми рисками
11. Инвестиции и управление активами
12. Учет операций с ценными бумагами и финансовыми активами

Пожалуйста, отнеси каждую тему к одной из этих категорий.
Требуется:
1. Общая тема: Укажите основную тему обсуждения. Приведите ключевые слова или категории, о чем идет речь.  
2. Что обсуждается: Описание конкретных аспектов, процессов, действий или деталей, связанных с указанной темой.

Инструкция:
- Если текст **содержит данные о бухгалтерии или деятельности предприятия**, верните извлеченную информацию в следующем формате:
  ```
  {
    "Тема": ["Категория 1", "Категория 2", ...],
    "Описание": ["Описание темы 1", "Описание темы 2", ...]
  }
  ```

- Если текст не связан с работой предприятия или содержит несущественные или ошибочные данные, верните:
  
```
  {"Тема": "",
    "Описание": ""
  }
  ```
Дополнительные требования:
- Если текст неоднозначен или содержит ошибки, извлеките максимально релевантную информацию, стараясь точно классифицировать тему.
- Игнорируйте неформальные комментарии, такие как обсуждение погоды, извинения или просьбы уточнить вопрос, если они не содержат полезной информации для анализа.
- В случае ошибок или некорректных данных в тексте, постарайтесь извлечь суть вопроса или темы и классифицировать ее как можно точнее.

Примеры:

Текст:  
"Мы обсуждали налоговые изменения и их влияние на бухгалтерию, а также подготовку отчетности за третий квартал." 
Ответ:
```
{
  "Тема": ["Налоги (налог на прибыль, НДС, налог на имущество, НДФЛ, социальные взносы, экологические налоги, таможенные пошлины, местные налоги)", "Финансовая отчетность (составление отчетности, баланс, отчет о прибылях и убытках, отчет о движении денежных средств, примечания, МСФО, консолидированная отчетность)"],
  "Описание": ["Влияние налоговых изменений на бухгалтерию", "Подготовка отчетности за третий квартал"]
}
```

Текст:  
"Погода сегодня отличная.*  
Ответ:  
```
{
  "Тема": "",
  "Описание": ""
}
```
"""

# Решение

In [ ]:
model_path = 'NousResearch/Hermes-3-Llama-3.1-8B'  #название модели https://huggingface.co/
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Блок квантования
# # # Configure 8-bit quantization
# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=True, # Используется 8-битное квантован
#     bnb_8bit_quant_type='nf4',  # 'nf4'
# )

# # # # Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False,  # Не используется 8-битное квантование
    load_in_4bit=True,   # Включаем 4-битное квантование
    bnb_4bit_quant_type='fp4',  # 4-битное квантование с плавающей точкой
)

# # Load model with quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.bfloat16},  # Использование bfloat16 в пайплайне
    device_map="auto",                             # Автоматическое распределение модели по устройствам
    pad_token_id=tokenizer.eos_token_id,
    truncation=True,
)

# Функция для инференса LLM
def interact(
    user_message,
    system_prompt=SYSTEM_PROMPT,
    temperature=0.3,
    max_length=1024
):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt")
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            temperature=temperature,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=pad_token_id
        )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [ ]:
# функция чтения файлов
def clean_text(text):
    cleaned_text = re.sub(r'[^а-яА-ЯЁё]', ' ', text)
    return cleaned_text
df = df.drop_duplicates(subset=['text'])
print(df.shape)
df = df[df['text'].apply(lambda row: len(clean_text(row).split()) > 10)]
df = df[~df['text'].str.strip().str.contains(r'^спасибо', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'^Если вы ищете работу, заполните форму', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'Напишите вакансию сюда, пожалуйста', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bПодпишитесь на канал\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bПодписка на каналы\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bИнвестируя в своё образование\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bпредставляю вам мини-курс\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bИщу подработку\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bИщу работу\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bблагодарю\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bпочему такой вопрос возник\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bНовым годом\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'(http[s]?://)' , case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'^Пожалуйста, задайте свой вопрос', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'^Пожалуйста, предоставьте текст', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bвопрос был другой\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'(https?://)', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bимеете ввиду?\b', case=False, na=False, regex=True)]
df = df[~df['text'].str.strip().str.contains(r'\bМожете уточнить\b', case=False, na=False, regex=True)]
print(df.shape)
df = df[df['text'].str.strip().str.contains(r'(?:\bПодскажите\b)|(?:\bподскажите\b)|(?:\bвопрос\b)|(?:\bВопрос\b)|(?:\bвопросы\b)', case=False, na=False, regex=True)]
print(df.shape)

In [ ]:
#Перемешаем, для того что бы не ждать когда все обработается, можно не делать
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df_dashbord =pd.DataFrame()
n = 0 
start_time = time.time()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:
                res = interact(row['text'].strip(), SYSTEM_PROMPT)
                cleaned_text = res.strip('```json\n').strip('```python\n').strip('\n```').replace('```', '').strip('```\n').strip()
                data = ast.literal_eval(cleaned_text)
                try:
                    data =pd.DataFrame(data)
                except:
                    data =pd.DataFrame([data])
        except Exception as e:
                print(f"Ошибка при обработке: {res}\n+++++++++++++++++++++++++\n{row['text']}\n---------------------------\n")
                data = {'Тема': '', 'Описание': ''}
                data =pd.DataFrame([data])
        try:
            data['id'] = row['id']
            data['date'] = row['date']
            data['user_id'] = row['user_id']
            data['text'] = row['text']
            data['messages'] = row['messages']

        except Exception as e:
            data = {'Тема': '', 'Описание': ''}
            data =pd.DataFrame([data])
            data['id'] = row['id']
            data['date'] = row['date']
            data['user_id'] = row['user_id']
            data['text'] = row['text']
            data['messages'] = row['messages']

            
        df_dashbord  = pd.concat([df_dashbord , data])
        n += 1
        if n%100 == 0:
            df_dashbord.to_csv('После_LLM_buch_dataset.csv', index=False)
            

df_dashbord = pd.DataFrame(data_list)

end_time = time.time()
elapsed_time = end_time - start_time
hours, remainder = divmod(elapsed_time, 3600)
minutes, seconds = divmod(remainder, 60)
print(f"Время выполнения: {int(hours)} часов, {int(minutes)} минут, {int(seconds)} секунд")

In [ ]:
df_dashbord.to_csv('После_LLM_buch_dataset.csv', index=False)